# Scraping Fangraphs Data

### Introduction

This notebook describes how to scrape historical baseball projection data from Fangraphs, to be used later to predict pitcher quality starts. I'll be scraping three years of projection data (2017, 2018, 2019) for each team, with each team-year of projection data on its own page. Each page of projection data has 2-3 tables with the statistics I want: "Pitchers, Counting Stats", "Pitchers, Rates and Averages", and "Pitchers, Assorted Other". To give an example, the Counting Stats table looks like this:

![](img/counting_stats_table.png)

### Retrieving Page Source Code

I’m going to use the requests library to send HTTP requests and retrieve the page source code. I’ll use BeautifulSoup to parse the HTML that comes back, along with re, the regular expressions library, to find and extract the data I care about. I’m also going to load pandas to put it all into dataframes after getting the data I need.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

I started by getting the list of URLS that I intended to scrape data from, taken from the links on this page (and similar pages for 2018 and 2019 projection data): https://blogs.fangraphs.com/category/2017-zips-projections/. 

In [2]:
list_urls_2017 = [
 'http://www.fangraphs.com/blogs/2017-zips-projections-baltimore-orioles',
 'http://www.fangraphs.com/blogs/2017-zips-projections-chicago-white-sox',
 'http://www.fangraphs.com/blogs/2017-zips-projections-houston-astros',
 'http://www.fangraphs.com/blogs/2017-zips-projections-boston-red-sox',
 'http://www.fangraphs.com/blogs/2017-zips-projections-cleveland-indians',
 'http://www.fangraphs.com/blogs/2017-zips-projections-los-angeles-angels',
 'http://www.fangraphs.com/blogs/2017-zips-projections-new-york-yankees',
 'http://www.fangraphs.com/blogs/2017-zips-projections-detroit-tigers',
 'http://www.fangraphs.com/blogs/2017-zips-projections-oakland-athletics',
 'http://www.fangraphs.com/blogs/2017-zips-projections-tampa-bay-rays',
 'http://www.fangraphs.com/blogs/2017-zips-projections-kansas-city-royals',
 'http://www.fangraphs.com/blogs/2017-zips-projections-seattle-mariners',
 'http://www.fangraphs.com/blogs/2017-zips-projections-toronto-blue-jays',
 'http://www.fangraphs.com/blogs/2017-zips-projections-minnesota-twins',
 'http://www.fangraphs.com/blogs/2017-zips-projections-texas-rangers',
 'http://www.fangraphs.com/blogs/2017-zips-projections-atlanta-braves',
 'http://www.fangraphs.com/blogs/2017-zips-projections-chicago-cubs',
 'http://www.fangraphs.com/blogs/2017-zips-projections-arizona-diamondbacks',
 'http://www.fangraphs.com/blogs/2017-zips-projections-miami-marlins',
 'http://www.fangraphs.com/blogs/2017-zips-projections-cincinnati-reds',
 'http://www.fangraphs.com/blogs/2017-zips-projections-colorado-rockies',
 'http://www.fangraphs.com/blogs/2017-zips-projections-new-york-mets',
 'http://www.fangraphs.com/blogs/2017-zips-projections-milwaukee-brewers',
 'http://www.fangraphs.com/blogs/2017-zips-projections-los-angeles-dodgers',
 'http://www.fangraphs.com/blogs/2017-zips-projections-philadelphia-phillies',
 'http://www.fangraphs.com/blogs/2017-zips-projections-pittsburgh-pirates',
 'http://www.fangraphs.com/blogs/2017-zips-projections-san-diego-padres',
 'http://www.fangraphs.com/blogs/2017-zips-projections-washington-nationals',
 'http://www.fangraphs.com/blogs/2017-zips-projections-st-louis-cardinals',
 'http://www.fangraphs.com/blogs/2017-zips-projections-san-francisco-giants']

list_urls_2018 = ['http://www.fangraphs.com/blogs/2018-zips-projections-baltimore-orioles',
 'http://www.fangraphs.com/blogs/2018-zips-projections-chicago-white-sox',
 'http://www.fangraphs.com/blogs/2018-zips-projections-houston-astros',
 'http://www.fangraphs.com/blogs/2018-zips-projections-boston-red-sox',
 'http://www.fangraphs.com/blogs/2018-zips-projections-cleveland-indians',
 'http://www.fangraphs.com/blogs/2018-zips-projections-los-angeles-angels',
 'http://www.fangraphs.com/blogs/2018-zips-projections-new-york-yankees',
 'http://www.fangraphs.com/blogs/2018-zips-projections-detroit-tigers',
 'http://www.fangraphs.com/blogs/2018-zips-projections-oakland-as',
 'http://www.fangraphs.com/blogs/2018-zips-projections-tampa-bay-rays',
 'http://www.fangraphs.com/blogs/2018-zips-projections-kansas-city-royals',
 'http://www.fangraphs.com/blogs/2018-zips-projections-seattle-mariners',
 'http://www.fangraphs.com/blogs/2018-zips-projections-toronto-blue-jays',
 'http://www.fangraphs.com/blogs/2018-zips-projections-minnesota-twins',
 'http://www.fangraphs.com/blogs/2018-zips-projections-texas-rangers',
 'http://www.fangraphs.com/blogs/2018-zips-projections-atlanta-braves',
 'http://www.fangraphs.com/blogs/2018-zips-projections-chicago-cubs',
 'http://www.fangraphs.com/blogs/2018-zips-projections-arizona-diamondbacks',
 'http://www.fangraphs.com/blogs/2018-zips-projections-miami-marlins',
 'http://www.fangraphs.com/blogs/2018-zips-projections-cincinnati-reds',
 'http://www.fangraphs.com/blogs/2018-zips-projections-colorado-rockies',
 'http://www.fangraphs.com/blogs/2018-zips-projections-new-york-mets',
 'http://www.fangraphs.com/blogs/2018-zips-projections-milwaukee-brewers',
 'http://www.fangraphs.com/blogs/2018-zips-projections-los-angeles-dodgers',
 'http://www.fangraphs.com/blogs/2018-zips-projections-philadelphia-phillies',
 'http://www.fangraphs.com/blogs/2018-zips-projections-pittsburgh-pirates',
 'http://www.fangraphs.com/blogs/2018-zips-projections-san-diego-padres',
 'http://www.fangraphs.com/blogs/2018-zips-projections-washington-nationals',
 'http://www.fangraphs.com/blogs/2018-zips-projections-st-louis-cardinals',
 'http://www.fangraphs.com/blogs/2018-zips-projections-san-francisco-giants']

list_urls_2019 = [
 'http://www.fangraphs.com/blogs/2019-zips-projections-baltimore-orioles',
 'http://www.fangraphs.com/blogs/2019-zips-projections-chicago-white-sox',
 'http://www.fangraphs.com/blogs/2019-zips-projection-houston-astros',
 'http://www.fangraphs.com/blogs/2019-zips-projections-boston-red-sox',
 'http://www.fangraphs.com/blogs/2019-zips-projections-cleveland-indians',
 'http://www.fangraphs.com/blogs/2019-zips-projections-los-angeles-angels',
 'http://www.fangraphs.com/blogs/2019-zips-projections-new-york-yankees',
 'http://www.fangraphs.com/blogs/2019-zips-projections-detroit-tigers',
 'http://www.fangraphs.com/blogs/2019-zips-projections-oakland-athletics',
 'http://www.fangraphs.com/blogs/2019-zips-projections-tampa-bay-rays',
 'http://www.fangraphs.com/blogs/2019-zips-projections-kansas-city-royals',
 'http://www.fangraphs.com/blogs/2019-zips-projections-seattle-mariners',
 'http://www.fangraphs.com/blogs/2019-zips-projections-toronto-blue-jays',
 'http://www.fangraphs.com/blogs/2019-zips-projections-minnesota-twins',
 'http://www.fangraphs.com/blogs/2019-zips-projections-texas-rangers',
 'http://www.fangraphs.com/blogs/2019-zips-projections-atlanta-braves',
 'http://www.fangraphs.com/blogs/2019-zips-projections-chicago-cubs',
 'http://www.fangraphs.com/blogs/2019-zips-projections-arizona-diamondbacks',
 'http://www.fangraphs.com/blogs/2019-zips-projections-miami-marlins',
 'http://www.fangraphs.com/blogs/2019-zips-projections-cincinnati-reds',
 'http://www.fangraphs.com/blogs/2019-zips-projections-colorado-rockies',
 'http://www.fangraphs.com/blogs/2019-zips-projections-new-york-mets',
 'http://www.fangraphs.com/blogs/2019-zips-projections-milwaukee-brewers',
 'http://www.fangraphs.com/blogs/2019-zips-projections-los-angeles-dodgers',
 'http://www.fangraphs.com/blogs/2019-zips-projections-philadelphia-phillies',
 'http://www.fangraphs.com/blogs/2019-zips-projections-pittsburgh-pirates',
 'http://www.fangraphs.com/blogs/2019-zips-projections-san-diego-padres',
 'http://www.fangraphs.com/blogs/2019-zips-projections-washington-nationals',
 'http://www.fangraphs.com/blogs/2019-zips-projections-st-louis-cardinals',
 'http://www.fangraphs.com/blogs/2019-zips-projections-san-francisco-giants']

For each page, I want to extract the source. I wrote this function that appends the source for each page into a list.

In [3]:
def get_data(urls):
    response = []
    for i in urls:
        response.append(requests.get(i))
    return response

I'll start with the 2017 data.

In [4]:
responses = get_data(list_urls_2017) 

Now that I have the source for all the 2017 ZiPS projections, I can write a function to parse them with BeautifulSoup.

In [5]:
def make_soup(response_list):
    soup = []
    for i in response_list:
        soup.append(BeautifulSoup(i.text, 'html5lib'))
    return soup

In [6]:
soups = make_soup(responses)

This soups object now contains the parsed source code for 2017 projection data for each team. 

### Retrieving Pitching Counts Table Data From Page Source Code

This function creates a series for each of the columns in the table, to be combined back into a table later. I'm sure there is a more elegant solution for this, but this works for my purposes.

In [7]:
def extract_column(number, data, upper_limit, num_cols):
    name = []
    name_list = []
    for i in range(0, upper_limit):
        name.append(number + i*num_cols)
    for i in name:
        name_list.append(data[i])
    return name_list

This function extracts the team name from the source, so it can be added to the resulting dataframe. 

In [8]:
def team_name(team_data, upper_limit):
    team_list = []
    team_clean = re.findall('[\s(A-z)]+[\|]', team_data)[0].replace('|', '').strip()
    for i in range(0, upper_limit):
        team_list.append(team_clean)
    return team_list

This function uses extract_column and team_name to generate the dataframes for each team's counting stats, for 2017. There's a lot in this function, and I want to highlight the .find() function, from BeautifulSoup. This is the function that identifies the table I want to extract data from in the source code, and takes values from those tables to create the series (using the extract_column function). 

In [9]:
def extract_pitching(soup_list, text_id, num_cols):
    pitch_list = []
    for j in soup_list:
        data_crude = j.find(text = text_id).findNext().text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        data = data_crude2[num_cols:]
        headers = data_crude2[:num_cols]
        headers.insert(1, "Team")
        team_data = j.find('title').text
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        team = team_name(team_data, upper_bound)
        throws = extract_column(1, data, upper_bound, num_cols)
        age = extract_column(2, data, upper_bound, num_cols)
        games = extract_column(3, data, upper_bound, num_cols)
        games_started = extract_column(4, data, upper_bound, num_cols)
        innings_pitched = extract_column(5, data, upper_bound, num_cols)
        strikeouts = extract_column(6, data, upper_bound, num_cols)
        walks = extract_column(7, data, upper_bound, num_cols)
        homeruns = extract_column(8, data, upper_bound, num_cols)
        hits = extract_column(9, data, upper_bound, num_cols)
        runs = extract_column(10, data, upper_bound, num_cols)
        earned_runs = extract_column(11, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           team,
                                           throws,
                                           age,
                                           games,
                                           games_started,
                                           innings_pitched,
                                           strikeouts,
                                           walks,
                                           homeruns,
                                           hits,
                                           runs,
                                           earned_runs)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

In [10]:
#Call function to get all pitching counting stats for 2017, table has 12 columns
pitch_data = extract_pitching(soups, "Pitchers, Counting Stats", 12)

### Retrieving Pitching Rates Table Data From Page Source Code

This function is similar to the one above, accounting for different columns in the pitching rates table.

In [11]:
def extract_pitching_rates(soup_list, text_id, num_cols):
    pitch_list = []
    for i in soup_list:
        data_crude = i.find(text = text_id).findNext().text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        headers = data_crude2[:num_cols]
        data = data_crude2[num_cols:]
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        IPs = extract_column(1, data, upper_bound, num_cols)
        TBFs = extract_column(2, data, upper_bound, num_cols)
        K_pcts = extract_column(3, data, upper_bound, num_cols)
        BB_pcts = extract_column(4, data, upper_bound, num_cols)
        BABIPs = extract_column(5, data, upper_bound, num_cols)
        ERAs = extract_column(6, data, upper_bound, num_cols)
        FIPs = extract_column(7, data, upper_bound, num_cols)
        ERA_minus = extract_column(8, data, upper_bound, num_cols)
        FIP_minus = extract_column(9, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           IPs,
                                           TBFs,
                                           K_pcts,
                                           BB_pcts,
                                           BABIPs,
                                           ERAs,
                                           FIPs,
                                           ERA_minus,
                                           FIP_minus)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

In [12]:
#Call function to get all pitching rate stats for 2017, table has 10 columns
pitch_rates_data = extract_pitching_rates(soups, "Pitchers, Rates and Averages", 10)

### Retrieving Pitching Other Table Data From Page Source Code

In [13]:
def extract_pitching_other(soup_list, text_id, num_cols):
    pitch_list = []
    for i in soup_list:
        data_crude = i.find(text = text_id).findNext().text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        headers = data_crude2[:num_cols]
        data = data_crude2[num_cols:]
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        IPs = extract_column(1, data, upper_bound, num_cols)
        K_9s = extract_column(2, data, upper_bound, num_cols)
        BB_9s = extract_column(3, data, upper_bound, num_cols)
        HR_9s = extract_column(4, data, upper_bound, num_cols)
        ERA_plus = extract_column(5, data, upper_bound, num_cols)
        zWARs = extract_column(6, data, upper_bound, num_cols)
        competition = extract_column(7, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           IPs,
                                           K_9s,
                                           BB_9s,
                                           HR_9s,
                                           ERA_plus,
                                           zWARs,
                                           competition)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

In [14]:
#Call function to get all other pitching stats for 2017, table has 8 columns
pitch_other_data = extract_pitching_other(soups, "Pitchers, Assorted Other", 8)

### Combine Counting, Rate, and Other Tables

In [15]:
pitch_rates_data.drop(columns = ['Player', 'IP'], inplace = True)
pitch_other_data.drop(columns = ['Player', 'IP', 'No. 1 Comp'], inplace = True)
#Start with counting and rates data
partial = pd.concat([pitch_data, pitch_rates_data], axis = 1)
#Add in other data
full = pd.concat([partial, pitch_other_data], axis = 1)

Now we can check the dataframe to make sure this is what we want.

In [16]:
#Check columns and preview dataframe
full.columns
full.head(6)

,Player,Team,T,Age,G,GS,IP,K,BB,HR,...,BABIP,ERA,FIP,ERA-,FIP-,K/9,BB/9,HR/9,ERA+,zWAR
0,Kevin Gausman,Baltimore Orioles,R,26,31,31,176.0,162,55,21,...,.297,3.94,3.82,92,88,8.28,2.81,1.07,104,2.8
1,Dylan Bundy,Baltimore Orioles,R,24,19,19,109.3,108,36,13,...,.298,3.70,3.75,86,86,8.89,2.96,1.07,110,2.1
2,Chris Tillman,Baltimore Orioles,R,29,29,29,167.0,133,60,21,...,.287,4.31,4.30,100,99,7.17,3.23,1.13,95,1.9
3,Zach Britton,Baltimore Orioles,L,29,66,0,64.0,73,18,5,...,.276,2.25,2.64,52,61,10.27,2.53,0.70,181,1.7
4,Mychal Givens,Baltimore Orioles,R,27,63,0,79.3,101,31,7,...,.298,2.95,3.03,69,70,11.46,3.52,0.79,138,1.4
5,Brad Brach,Baltimore Orioles,R,31,63,0,74.0,87,29,8,...,.278,3.04,3.25,71,75,10.58,3.53,0.97,134,1.2


This looks good! I'll save this as a csv file, so it can be easily loaded later. 

In [17]:
# full.to_csv('projected_2017.csv', index = False)

### Rerunning to Scrape and Process 2018 Data

Working with the 2018 data is pretty straightforward, now that we've processed the data from 2017.

In [18]:
responses = get_data(list_urls_2018) 
soups = make_soup(responses)
pitch_data = extract_pitching(soups, "Pitchers, Counting Stats", 12)
pitch_rates_data = extract_pitching_rates(soups, "Pitchers, Rates and Averages", 10)
pitch_other_data = extract_pitching_other(soups, "Pitchers, Assorted Other", 8)
pitch_rates_data.drop(columns = ['Player', 'IP'], inplace = True)
pitch_other_data.drop(columns = ['Player', 'IP', 'No. 1 Comp'], inplace = True)
partial = pd.concat([pitch_data, pitch_rates_data], axis = 1)
full = pd.concat([partial, pitch_other_data], axis = 1)
#full.to_csv('projected_2018.csv', index = False)

### Rerunning to Scrape and Process 2019 Data

2019 is less straightforward. The tables are laid out differently, and one of the teams has one less column.

In [19]:
responses = get_data(list_urls_2019) 
soups = make_soup(responses)

In [20]:
#Function to get pitching counting stats. Many more columns present than in 2017/2018.
def extract_pitching_2019(soup_list, text_id, num_cols):
    pitch_list = []
    for j in soup_list:
        data_crude = j.find(text = text_id).findNext().text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        data = data_crude2[num_cols:]
        headers = data_crude2[:num_cols]
        headers.insert(1, "Team")
        team_data = j.find('title').text
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        team = team_name(team_data, upper_bound)
        throws = extract_column(1, data, upper_bound, num_cols)
        age = extract_column(2, data, upper_bound, num_cols)
        wins = extract_column(3, data, upper_bound, num_cols)
        losses = extract_column(4, data, upper_bound, num_cols)
        ERA = extract_column(5, data, upper_bound, num_cols)
        games = extract_column(6, data, upper_bound, num_cols)
        games_started = extract_column(7, data, upper_bound, num_cols)
        innings_pitched = extract_column(8, data, upper_bound, num_cols)
        hits = extract_column(9, data, upper_bound, num_cols)
        earned_runs = extract_column(10, data, upper_bound, num_cols)
        homeruns = extract_column(11, data, upper_bound, num_cols)
        walks = extract_column(12, data, upper_bound, num_cols)
        strikeouts = extract_column(13, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           team,
                                           throws,
                                           age,
                                           wins, 
                                           losses, 
                                           ERA, 
                                           games, 
                                           games_started, 
                                           innings_pitched,
                                           hits, 
                                           earned_runs, 
                                           homeruns, 
                                           walks, 
                                           strikeouts)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

In [21]:
#Call function to get all pitching counting stats for 2019, table has 14 columns
pitch_data = extract_pitching_2019(soups, 'Pitchers – Counting Stats', 14)

In [22]:
#Function to get pitching rates. Note this doesn't work for one team's data.
def extract_pitching_rates_2019(soup_list, text_id, num_cols):
    pitch_list = []
    for j in soup_list:
        data_crude = j.find(text = text_id).parent.parent.parent.text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        data = data_crude2[num_cols:]
        headers = data_crude2[:num_cols]
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        IPs = extract_column(1, data, upper_bound, num_cols)
        TBFs = extract_column(2, data, upper_bound, num_cols)
        K_9s = extract_column(3, data, upper_bound, num_cols)
        BB_9s = extract_column(4, data, upper_bound, num_cols)
        HR_9s = extract_column(5, data, upper_bound, num_cols)
        BABIPs = extract_column(6, data, upper_bound, num_cols)
        ERA_plus = extract_column(5, data, upper_bound, num_cols)
        ERA_minus = extract_column(8, data, upper_bound, num_cols)
        FIPs = extract_column(9, data, upper_bound, num_cols)
        WAR = extract_column(10, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           IPs,
                                           TBFs,
                                           K_9s,
                                           BB_9s,
                                           HR_9s,
                                           BABIPs,
                                           ERA_plus,
                                           ERA_minus,                                           
                                           FIPs,                                        
                                           WAR)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

One of the teams has one fewer columns than the others, so I will remove it from the list and process it separately.

In [23]:
new_soup = soups[0:4] + soups[5:] 

Now I can get all of the pitching rate stats (except for the one rogue team).

In [24]:
#Call function to get all (but one team's) pitching rate stats for 2019, table has 11 columns
pitch_rates_data = extract_pitching_rates_2019(new_soup, 'TBF', 11)

The code below is modified to account for the one team that has one less column than the others.

In [25]:
#Function to get pitching rates. Note this is particular for one team's data. Doesn't
#contain ERA_minus data.
def extract_pitching_rates_2019_4(soup_list, text_id, num_cols):
    pitch_list = []
    for j in soup_list:
        data_crude = j.find(text = text_id).parent.parent.parent.text.split("\n")
        data_crude2 = []
        for i in data_crude:
            if i != '':
                data_crude2.append(i)
        data = data_crude2[num_cols:]
        headers = data_crude2[:num_cols]
        upper_bound = int(len(data)/num_cols)
        names = extract_column(0, data, upper_bound, num_cols)
        IPs = extract_column(1, data, upper_bound, num_cols)
        TBFs = extract_column(2, data, upper_bound, num_cols)
        K_9s = extract_column(3, data, upper_bound, num_cols)
        BB_9s = extract_column(4, data, upper_bound, num_cols)
        HR_9s = extract_column(5, data, upper_bound, num_cols)
        BABIPs = extract_column(6, data, upper_bound, num_cols)
        ERA_plus = extract_column(7, data, upper_bound, num_cols)
        FIPs = extract_column(8, data, upper_bound, num_cols)
        WAR = extract_column(9, data, upper_bound, num_cols)
        pitch_data = pd.DataFrame(list(zip(names, 
                                           IPs,
                                           TBFs,
                                           K_9s,
                                           BB_9s,
                                           HR_9s,
                                           BABIPs,
                                           ERA_plus,                                     
                                           FIPs,                                        
                                           WAR)),
                                 columns = headers)
        pitch_list.append(pitch_data)
    pitching = pd.concat(pitch_list, ignore_index = True, axis = 0)
    return pitching

In [26]:
#Work with just the one team's data that has a different number of columns
soups4 = [soups[4]]

In [27]:
#Call function to get one team's pitching rate stats for 2019, table has 10 columns
pitch_rates_data_4 = extract_pitching_rates_2019_4(soups4, 'TBF', 10)

Now I can splice all of the team data back together, remove redundant and unnecessary columns, and reorder as necessary.

In [28]:
#splicing the one team's data back in
pitch_rates_data_p1 = pitch_rates_data.iloc[0:175]
pitch_rates_data_p3 = pitch_rates_data.iloc[175:]
pitch_rates_combined = pd.concat([pitch_rates_data_p1, pitch_rates_data_4, pitch_rates_data_p3], ignore_index = True)

In [29]:
#removing redundant columns and No.1 Comp column.
pitch_rates_combined.drop(columns = ['Player', 'No. 1 Comp'], inplace = True)
full_2019 = pd.concat([pitch_data, pitch_rates_combined], axis = 1)
#Reordering dataframe to match 2017-2018 column order
projected_2019 = full_2019[['Player', 'Team', 'T', 'Age', 'G', 'GS', 'IP', 'SO', 
                            'BB', 'HR', 'H', 'ER', 'TBF', 'BABIP', 'ERA', 'FIP', 
                            'ERA-', 'K/9', 'BB/9', 'HR/9', 'ERA+', 'WAR', 'W', 'L']]

In [ ]:
# Print full 2018 pitching data to csv, 
# commented out so datafile is not overwritten each time code is run
#projected_2019.to_csv('projected_2019.csv', index = False)

### Wrap-up

At the end of of this, we have three years of projection data for pitching. This can all be done in Beautiful Soup, because all of the data I need is already visible on the page once it is rendered. To get season data, I scrape regular season data from Baseball Reference that uses Javascript, which requires using Selenium (covered in the Baseball Reference Notebook in the repo). 